In [9]:
import numpy as np
import pandas as pd
from datetime import datetime

In [10]:
df_bill = pd.read_csv('bill.csv')
# print(df_bill)

df_payment = pd.read_csv('payment.csv')
# print(df_payment["Pending"])

In [11]:
df_payment["Date"] = pd.to_datetime(df_payment.Date)
df_bill["Date"] = pd.to_datetime(df_bill.Date)
sorted_payment = df_payment.sort_values('Date')

In [12]:
for i, row in df_bill.iterrows():
    tax = 0;
    if row["Date"] <= datetime.strptime('31-03-2012', '%d-%m-%Y'):
        df_bill.set_value(i, "tax", 10.30)
        tax = 10.30
        
    if (row["Date"] >= datetime.strptime('01-04-2012', '%d-%m-%Y')) and (row["Date"] <= datetime.strptime('31-05-2015', '%d-%m-%Y')):
        df_bill.set_value(i, "tax", 12.36)
        tax = 12.36
        
    if (row["Date"] >= datetime.strptime('01-06-2015', '%d-%m-%Y')) and (row["Date"] <= datetime.strptime('15-11-2015', '%d-%m-%Y')):
        df_bill.set_value(i, "tax", 14.00)
        tax = 14.00
        
    if (row["Date"] >= datetime.strptime('16-11-2015', '%d-%m-%Y')) and (row["Date"] <= datetime.strptime('31-05-2016', '%d-%m-%Y')):
        df_bill.set_value(i, "tax", 14.50)
        tax = 15.00
        
    if (row["Date"] >= datetime.strptime('01-06-2016', '%d-%m-%Y')) and (row["Date"] <= datetime.strptime('30-06-2017', '%d-%m-%Y')):
        df_bill.set_value(i, "tax", 15.00)
        tax = 15.00
        
    if (row["Date"] >= datetime.strptime('01-07-2017', '%d-%m-%Y')):
        df_bill.set_value(i, "tax", 18.00)
        tax = 18.00
        
    tax_amount = row["Amount"] * (tax) / 100   
    base_amount = row["Amount"] - tax_amount

    df_bill.set_value(i, "tax", tax)
    df_bill.set_value(i, "tax_amount", tax_amount)
    df_bill.set_value(i, "base_amount", base_amount)
    
    # TDS on base value @ 2%
    tds = row["Amount"] * 2/100
    df_bill.set_value(i, "tds", tds)
    
    # Invoice amount after TDS 
    amount_afte_tds = row["Amount"] - tds
    df_bill.set_value(i, "amount_after_tds", amount_afte_tds)
    
print(df_bill[df_bill.columns[[1,4,5,6,7, 8, 9]]].head(5))

        Date    Amount   tax  tax_amount  base_amount     tds  \
0 2015-03-31    207.56 12.36       25.65       181.91    4.15   
1 2016-04-02 344426.00 15.00    51663.90    292762.10 6888.52   
2 2016-04-30    649.00 15.00       97.35       551.65   12.98   
3 2016-04-30 119955.00 15.00    17993.25    101961.75 2399.10   
4 2016-05-31    963.00 15.00      144.45       818.55   19.26   

   amount_after_tds  
0            203.41  
1         337537.48  
2            636.02  
3         117555.90  
4            943.74  


In [13]:
payment = 0

# Loop over each payment
for index, row in sorted_payment.iterrows():
   
    
    # Holds the total value paid by one payment
    total_paid = 0
    
    # Holds total tds paid per payment
    total_tds = 0
    
    # Sum of invoice amount
    sum_of_invoice_amount = 0
    
    
    # Sum of amount after TDS
    sum_of_amount_after_tds = 0
    
    # Amount of the payment
    payment = int(row["Pending"])
    
    print("Payment of: ",row["Pending"], "was made on", row["Date"])
    print("Total payment including any prev balance:", payment)
    print("--------------------------------------")

    # Get all the bills before the date of this payment and which are not already paid
    # if payment_id column in the sheet is null, that bill is not paid
    bills_before_date = df_bill.loc[(df_bill["Date"] <= row["Date"]) & (df_bill["payment_id"]).isnull()]
    
    # Sort the bill according to the amount_after_tds
    # This makes easy to get the bills which fall under the payment we have
    sorted_bill = bills_before_date.sort_values("amount_after_tds")
    
    # List to hold all the bill paid with this payment
    # Holds the bill_id
    # Used to update the Payment Sheet
    bills = []
    
    
    # Loop over the sorted_bill 
    for i, bill in sorted_bill.iterrows():
        
        # check if the bill amount is less then the payment amount we have
        if bill["amount_after_tds"] <= payment:
            
            print("Paid: ", bill["amount_after_tds"])
            
            # Add the paid amount to the total_paid tally
            total_paid = total_paid + bill["amount_after_tds"]
            
            # The payment we have is reduced by the amount we just paid
            payment = payment - bill["amount_after_tds"]
            
            # Add tds of this bill to the total tds paid by this payment
            total_tds += bill["tds"]
            
            # Add the invoice amount of this bill to Sum of Invoice amount 
            sum_of_invoice_amount += bill["Amount"]
            
            # Add the after_tds amount of this bill to the Sum of Amount after TDS
            sum_of_amount_after_tds += bill["amount_after_tds"]
            
            # Add this bill_id to the list
            bills.append(str(bill["bill_id"]))
            
            # Set the payment_id of the bill to be this payment_id
            df_bill.set_value(i, "payment_id", row["payment_id"])
            
    
    # convert the list to string
    # this is to store it in the sheet
    # Idk why they don't store list object directly to sheet
    # will research later
    # this works fine
    bills_string =  ",".join(bills)

    # Update the payment dataframe with the bill ids paid by this payment
    df_payment.set_value(index, "bill_id", bills_string)
    
    # Set the sum of invoice amount of bills paid by this payment
    df_payment.set_value(index, "Sum of Inv Amount", sum_of_invoice_amount if len(bills)>0 else 0)
    
    # Update total tds paid by the payment on the base dataframe
    df_payment.set_value(index, "total_tds", total_tds if len(bills)>0 else 0)
    
    # Set the sum of amount after tds of bills paid by this payment
    df_payment.set_value(index, "Sum of amount_after_tds", sum_of_amount_after_tds if len(bills)>0 else 0)
    
    # We can see that, One payment can't be fully utilized to pay the bill
    # there will be some residue left which can't be used to pay further bills
    # Maybe because they are less as compared to the amount of each bill left to be paid
    # There is a residue column in the payment sheet 
    # add this residue to that column and forget about it
    # This will not be used in the next payment
    df_payment.set_value(index, "residue", payment if len(bills)>0 else 0)
    
    print("Paid:", ",".join(bills))
    print("---------------------------------------")
    print("Total Paid:", total_paid)
    print("What was left: ", payment)
    print("_______________________________________")

Payment of:  1894 was made on 2016-10-31 00:00:00
Total payment including any prev balance: 1894
--------------------------------------
Paid:  203.4088
Paid:  636.02
Paid:  943.74
Paid: 1,3,5
---------------------------------------
Total Paid: 1783.1688
What was left:  110.83120000000008
_______________________________________
Payment of:  308 was made on 2016-10-31 00:00:00
Total payment including any prev balance: 308
--------------------------------------
Paid: 
---------------------------------------
Total Paid: 0
What was left:  308
_______________________________________
Payment of:  6577 was made on 2017-02-16 00:00:00
Total payment including any prev balance: 6577
--------------------------------------
Paid:  4292.4
Paid: 9
---------------------------------------
Total Paid: 4292.4
What was left:  2284.6000000000004
_______________________________________
Payment of:  694722 was made on 2017-03-30 00:00:00
Total payment including any prev balance: 694722
-----------------------

In [14]:
# print(df_payment)
# df_payment.to_csv("payment_11.csv", sep='\t')
writer = pd.ExcelWriter('entire_value_at_2_percentage.xlsx')
df_bill.to_excel(writer,'bill')
df_payment.to_excel(writer,'payment')

pd.options.display.float_format = "{:.2f}".format
df_summary = df_payment.loc[:, ["payment_id", "Sum of Inv Amount", "total_tds", "Sum of amount_after_tds"]]
df_summary = df_summary[df_summary["Sum of Inv Amount"] > 0]
sum_of_inv_amount = df_summary["Sum of Inv Amount"].sum()
sum_of_tds = df_summary["total_tds"].sum()
sum_of_amount_after_tds = df_summary["Sum of amount_after_tds"].sum()
df_with_summary = df_summary.append({'payment_id':'Grand Total', 'Sum of Inv Amount': sum_of_inv_amount, 'total_tds':sum_of_tds, 'Sum of amount_after_tds': sum_of_amount_after_tds}, ignore_index=True)
df_with_summary.to_excel(writer, 'summary')


writer.save()